In [8]:
import psycopg2
import secrets
import pandas as pd
import os
import shapely
import datetime
import subprocess
from census import Census 
import requests
import folium

In [9]:
db_name = 'tweets_paul'
user = 'paul'
password = secrets.password_paul
host = '74.207.246.217'
connstring = "dbname={} user={} host={} password={}".format(db_name, user, host, password)
conn = psycopg2.connect(database=db_name, 
                        user=user,
                        password=password,
                        host=host)

In [10]:
cur = conn.cursor()

# Check data types
cur.execute("SELECT column_name, data_type FROM information_schema.columns \
            WHERE table_name = 'tweets';")
cur.fetchall()

[('index', 'bigint'),
 ('id', 'bigint'),
 ('lat', 'double precision'),
 ('lng', 'double precision'),
 ('text', 'text'),
 ('timeStamp', 'timestamp without time zone'),
 ('user_id', 'bigint'),
 ('location', 'USER-DEFINED')]

In [5]:
# bash command to insert shapefile into postgis database

"""
shp2pgsql -I -W 'latin1' -s 4326 /home/paul/Documents/Coursework/tweet-db/data/tl_2010_06_county10/tl_2010_06_county10.shp counties | psql -h 74.207.246.217 -d tweets_paul -U paul
"""

# Check data types in the counties 
cur.execute("SELECT column_name, data_type FROM information_schema.columns \
            WHERE table_name = 'counties';")
cur.fetchall()

[('gid', 'integer'),
 ('statefp10', 'character varying'),
 ('countyfp10', 'character varying'),
 ('countyns10', 'character varying'),
 ('geoid10', 'character varying'),
 ('name10', 'character varying'),
 ('namelsad10', 'character varying'),
 ('lsad10', 'character varying'),
 ('classfp10', 'character varying'),
 ('mtfcc10', 'character varying'),
 ('csafp10', 'character varying'),
 ('cbsafp10', 'character varying'),
 ('metdivfp10', 'character varying'),
 ('funcstat10', 'character varying'),
 ('aland10', 'double precision'),
 ('awater10', 'double precision'),
 ('intptlat10', 'character varying'),
 ('intptlon10', 'character varying'),
 ('geom', 'USER-DEFINED')]

In [6]:
# Check the first few FIPS codes; looks good

cur.execute("SELECT geoid10 FROM counties LIMIT 5;")
cur.fetchall()

[('06059',), ('06103',), ('06011',), ('06083',), ('06051',)]

In [7]:
# Calculate the number of tweets inside of Contra Costa County.

query = "SELECT count(*) FROM counties, tweets \
         WHERE counties.geoid10='06013' \
         AND ST_Intersects(counties.geom, tweets.location);"

cur.execute(query)
cur.fetchall()

[(8502L,)]

In [14]:
# How many Tweets fall 100 miles outside of Alameda County? (i.e., fall outside of a 100 mile polygon
# surrounding the Alameda County).

# This is the complement

query = "SELECT count(*) FROM counties, tweets \
        WHERE counties.geoid10='06001' \
        AND ST_Dwithin(ST_Transform(counties.geom, 3157), ST_Transform(tweets.location, 3157), 160934);"

cur.execute(query)
cur.fetchall()

[(85094L,)]

In [15]:
# How many Tweets fall 100 miles outside of Alameda County? (i.e., fall outside of a 100 mile polygon
# surrounding the Alameda County).

# Fixed to answer the question by adding NOT to query

query = "SELECT count(*) FROM counties, tweets \
        WHERE counties.geoid10='06001' \
        AND NOT ST_Dwithin(ST_Transform(counties.geom, 3157), ST_Transform(tweets.location, 3157), 160934);"

cur.execute(query)
cur.fetchall()

[(14906L,)]

In [36]:
# Get the population data for California counties directly from the Census API 

c = Census(secrets.censuskey)

population = c.sf1.get('P0010001', geo={'for': 'county:*',
                                       'in': 'state:06'})
pop = pd.DataFrame(population)
pop.columns = ['population', 'county', 'state']
pop['geoid10'] = pop.state + pop.county
pop.head()

,population,county,state,geoid10
0,1510271,001,06,06001
1,1175,003,06,06003
2,38091,005,06,06005
3,220000,007,06,06007
4,45578,009,06,06009


In [37]:
# Duplicate SQL connection using sqlalchemy to take advantage of 
# pandas built in "to_sql" method

from sqlalchemy import create_engine
enginestring = 'postgresql://{}:{}@74.207.246.217:5432/tweets_paul'.format(user, password)
engine = create_engine(enginestring)

# Load data to SQL
pop.to_sql('population', engine)

# Commit changes
conn.commit()

In [44]:
# Check data types for the new population table
cur.execute("SELECT column_name, data_type FROM information_schema.columns \
            WHERE table_name = 'population';")
cur.fetchall()

[('index', 'bigint'),
 ('population', 'text'),
 ('county', 'text'),
 ('state', 'text'),
 ('geoid10', 'text')]

In [22]:
# provide a visualization of tweets per capita for California counties

cur.execute("SELECT counties.name10, population.population FROM population INNER JOIN counties ON counties.geoid10 = population.geoid10;")
cur.fetchall()

[('Orange', '3010232'),
 ('Tehama', '63463'),
 ('Colusa', '21419'),
 ('Santa Barbara', '423895'),
 ('Mono', '14202'),
 ('Monterey', '415057'),
 ('Placer', '348432'),
 ('Amador', '38091'),
 ('Calaveras', '45578'),
 ('Imperial', '174528'),
 ('Siskiyou', '44900'),
 ('Sonoma', '483878'),
 ('Santa Clara', '1781642'),
 ('Kern', '839631'),
 ('Yolo', '200849'),
 ('Mendocino', '87841'),
 ('Sacramento', '1418788'),
 ('Madera', '150865'),
 ('Yuba', '72155'),
 ('Tulare', '442179'),
 ('San Diego', '3095313'),
 ('Plumas', '20007'),
 ('San Benito', '55269'),
 ('Shasta', '177223'),
 ('Stanislaus', '514453'),
 ('Mariposa', '18251'),
 ('Fresno', '930450'),
 ('Alpine', '1175'),
 ('Marin', '252409'),
 ('Glenn', '28122'),
 ('Lassen', '34895'),
 ('Del Norte', '28610'),
 ('Napa', '136484'),
 ('San Luis Obispo', '269637'),
 ('San Mateo', '718451'),
 ('Nevada', '98764'),
 ('San Joaquin', '685306'),
 ('San Bernardino', '2035210'),
 ('Sutter', '94737'),
 ('Riverside', '2189641'),
 ('Trinity', '13786'),
 ('Contra

In [42]:
cur.execute("SELECT counties.name10 as name, count(*) as count_tweets FROM tweets, counties \
             WHERE ST_Intersects(tweets.location, counties.geom) \
             GROUP BY counties.name10;")
cur.fetchall()

[('Orange', 742L),
 ('El Dorado', 83L),
 ('Napa', 492L),
 ('Mendocino', 286L),
 ('Plumas', 53L),
 ('Santa Clara', 19531L),
 ('San Diego', 841L),
 ('Nevada', 61L),
 ('Sutter', 479L),
 ('San Luis Obispo', 530L),
 ('Los Angeles', 3946L),
 ('Kings', 7L),
 ('San Benito', 158L),
 ('Del Norte', 79L),
 ('Madera', 190L),
 ('Santa Barbara', 76L),
 ('Placer', 72L),
 ('Yolo', 490L),
 ('Stanislaus', 1077L),
 ('Imperial', 557L),
 ('Riverside', 2270L),
 ('Amador', 73L),
 ('Tehama', 268L),
 ('San Mateo', 8341L),
 ('Solano', 1543L),
 ('Ventura', 1049L),
 ('Alpine', 2L),
 ('Siskiyou', 61L),
 ('Fresno', 415L),
 ('Lassen', 21L),
 ('Monterey', 662L),
 ('San Joaquin', 1146L),
 ('Kern', 47L),
 ('Butte', 419L),
 ('Glenn', 304L),
 ('San Bernardino', 2664L),
 ('Yuba', 71L),
 ('Tuolumne', 45L),
 ('Contra Costa', 8502L),
 ('San Francisco', 17219L),
 ('Shasta', 312L),
 ('Alameda', 17045L),
 ('Sonoma', 692L),
 ('Merced', 1075L),
 ('Santa Cruz', 2695L),
 ('Sacramento', 2195L),
 ('Humboldt', 75L),
 ('Calaveras', 10L)

In [11]:
query = """
SELECT 

county_pop.name, 
county_tweets.count_tweets::float/county_pop.pop::float

FROM

    (SELECT counties.name10 as name, population.population as pop, counties.geom as geom 
    FROM population 
    INNER JOIN counties 
    ON counties.geoid10 = population.geoid10) county_pop
    
    INNER JOIN

    (SELECT counties.name10 as name, count(*) as count_tweets 
    FROM tweets, counties
    WHERE ST_Intersects(tweets.location, counties.geom)
    GROUP BY counties.name10) county_tweets

ON county_pop.name = county_tweets.name;

"""

cur.execute(query)
result = cur.fetchall()

In [12]:
df = pd.DataFrame(result, columns=['NAME10', 'tweets_per_capita'])

In [13]:
df.head()

,NAME10,tweets_per_capita
0,Orange,0.000246
1,Tehama,0.004223
2,Santa Barbara,0.000179
3,Monterey,0.001595
4,Placer,0.000207


In [14]:
# To visualize with Folium, we need to add counties where there are no tweets.
# Start with getting all the counties from the database.

query = """
SELECT NAME10 FROM counties;
"""

cur.execute(query)
result = cur.fetchall()
counties = pd.DataFrame(result, columns=['NAME10'])
counties.head()

,NAME10
0,Orange
1,Tehama
2,Colusa
3,Santa Barbara
4,Mono


In [15]:
# Merge missing counties into DataFrame

county_tweets = pd.merge(counties, df, how='left')
county_tweets.head()

,NAME10,tweets_per_capita
0,Orange,0.000246
1,Tehama,0.004223
2,Colusa,NaN
3,Santa Barbara,0.000179
4,Mono,NaN


In [ ]:
# Turn into tweets per 1,000,000 people for easier interpretation

county_tweets.tweets_per_capita = county_tweets.tweets_per_capita * 1000000
county_tweets.to_csv(os.path.join(os.getcwd(),'..','data','tweets_per_capita.csv'), index=False)

In [44]:
county_tweets

,NAME10,tweets_per_capita
0,Orange,246.492629
1,Tehama,4222.933048
2,Colusa,NaN
3,Santa Barbara,179.289683
4,Mono,NaN
5,Monterey,1594.961656
6,Placer,206.640033
7,Amador,1916.463207
8,Calaveras,219.404098
9,Imperial,3191.464980


In [18]:
# Bash command to turn county shapefile into geoJSON for visualization

"""
ogr2ogr -f GeoJSON -t_srs crs:84 counties.geojson tl_2010_06_county10.shp
"""

'\nogr2ogr -f GeoJSON -t_srs crs:84 counties.geojson tl_2010_06_county10.shp\n'

In [22]:
geojson = os.path.join(os.getcwd(), '..', 'data', 'tl_2010_06_county10', 'counties.geojson')

In [47]:
county_map = folium.Map(location=[38, -119], zoom_start=6, tiles="Mapbox Bright") 
county_map.choropleth(geo_path=geojson,
               data=pd.read_csv(os.path.join(os.getcwd(),'..','data','tweets_per_capita.csv')),
               columns= ['NAME10', 'tweets_per_capita'], 
               key_on='feature.properties.NAME10',
               fill_color='YlGn',
               fill_opacity=0.7,
               line_opacity=0.2,
               legend_name="Tweets per 1M people")

county_map.save('counties.html')

/home/paul/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:9: FutureWarning: 'threshold_scale' default behavior has changed. Now you get a linear scale between the 'min' and the 'max' of your data. To get former behavior, use folium.utilities.split_six.


In [48]:
%%HTML
<iframe width="70%" height="700" src="counties.html"></iframe>

In [72]:
conn.rollback()

In [28]:
conn.close()